In [3]:
!nvidia-smi

Sat Nov 28 19:21:22 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN RTX           Off  | 00000000:04:00.0 Off |                  N/A |
| 41%   39C    P2    68W / 280W |   2754MiB / 24220MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  TITAN RTX           Off  | 00000000:05:00.0 Off |                  N/A |
| 41%   51C    P8    25W / 280W |     11MiB / 24220MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [4]:
######### IMPORTING NECESSARY MODULES #########
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset
import sys
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

sys.path.append('/home/ironman/abhishek/AliProducts/architectures/pytorch-image-models/timm/models/')
sys.path.append('/home/ironman/abhishek/AliProducts/architectures/pytorch-image-models/')
sys.path.append('/home/ironman/abhishek/AliProducts/Helper/')

from dataloader import mydataset, create_prime_dict 
from trainer_ViT import train, test_classify, eval_classify
from Load_model import load



from vision_transformer import vit_base_patch32_384

In [5]:
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:2


**Dataloading Scheme**

In [6]:
trainlist = '/home/ironman/abhishek/AliProducts/Newlist/NEW_train_list1.txt'
validlist = '/home/ironman/abhishek/AliProducts/Newlist/NEW_valid_list1.txt'

In [7]:
prime_dict = create_prime_dict(trainlist)

Number of classes =  50030


In [8]:
#### Train Dataloader #### 
train_dataset = mydataset(trainlist, prime_dict, name='train')          
train_dataloader = data.DataLoader(train_dataset, shuffle= True, batch_size = 256, num_workers=16,pin_memory=True)


#### Validation Dataloader #### 
validation_dataset = mydataset(validlist, prime_dict, name='valid')         
validation_dataloader = data.DataLoader(validation_dataset, shuffle=False, batch_size = 64, num_workers=16,pin_memory=True)

**Model Definition**

In [9]:
model = vit_base_patch32_384(pretrained=True)


model = nn.DataParallel(model,device_ids=[2,3]).to(device)

In [10]:
model

DataParallel(
  (module): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32))
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks): ModuleList(
      (0): Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU()
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
        )
      )
      (1): Block(
        (norm1): LayerNorm((768

In [11]:
'''
Transfer Learning
'''

# for param in model.module.parameters():
#     param.requires_grad = False

fc_inputs = model.module.head.in_features

model.module.head = nn.Sequential(
    nn.Linear(fc_inputs, 1000),
#     nn.BatchNorm1d(4096),
    nn.GELU(),
    nn.Linear(1000, 50030)
    
)

model.to(device)

DataParallel(
  (module): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32))
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks): ModuleList(
      (0): Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU()
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
        )
      )
      (1): Block(
        (norm1): LayerNorm((768

**Hyperparameters**

In [12]:
## Loss Function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.05, weight_decay=1e-4, momentum=0.9)


# Epochs
num_Epochs = 30

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= 10, gamma = 0.1)

#Cutmix
# beta = 1
# cutmix_prob = 1

In [14]:
modelname = 'ViT_pretrained_vit_base_patch32_384_mlpadded'
modelpath = '/home/ironman/abhishek/saved_model_checkpoints/AliProducts/' + modelname

In [ ]:
writer = SummaryWriter(modelname)

train(model, train_dataloader, validation_dataloader, criterion, optimizer, lr_scheduler, modelpath, writer, device, epochs = num_Epochs)

writer.flush()
writer.close()

loss 10.82594822883606
loss 21.63555132865906
loss 32.40773811340332
loss 43.06737935066223
loss 53.53862006187439
loss 63.73400384902954
loss 73.58893639564513
loss 83.05676627159119
loss 92.13545754432678
loss 100.79769291877747
loss 108.99147634506225
loss 116.72287644386292
loss 124.00898491382598
loss 130.83184237480162
loss 137.24160254478454
loss 143.31865582942962
loss 149.03403333187103
loss 154.37838282108308
loss 159.43289021015167
loss 164.2814518022537
loss 168.79427546977996
loss 173.2109024143219
loss 177.3862962436676
loss 181.48733843803404
loss 185.34483425855638
loss 189.1539590716362
loss 192.84520596265793
loss 196.45930205106734
loss 199.97080763816834
loss 203.39186331033707
loss 206.73034244298935
loss 210.056429977417
loss 213.32372375249864
loss 216.51953795671463
loss 219.65635427236558
loss 222.75146915435792
loss 225.77938162326814
loss 228.7580301785469
loss 231.67602324962616
loss 234.5677730154991
loss 237.42340968370436
loss 240.2565420269966
loss 243.0

loss 2.4164986884593964
loss 3.2217390370368957
loss 4.034864583015442
loss 4.88416322350502
loss 5.73066682100296
loss 6.611034525036811
loss 7.513768311142922
loss 8.44815440773964
loss 9.371645398736
loss 10.329968422651291
loss 11.315584715008736
loss 12.299922425150871
loss 13.32246769964695
loss 14.37076464176178
loss 15.414580703377723
loss 16.504852572083472
loss 17.57747867822647
loss 18.655234323143958
loss 19.774667590856552
loss 20.929778911471367
loss 22.08407855987549
loss 23.24386607706547
loss 24.452331691384316
loss 25.64300217926502
loss 26.882636727690695
loss 28.13981659770012
loss 29.41103259563446
loss 30.668233798742293
loss 31.940993430018423
loss 33.23389583587647
loss 34.554469304084776
loss 35.8578355884552
loss 37.17195714771748
loss 38.54716098189354
loss 39.8916356754303
loss 41.28657398700714
loss 42.70878590643406
loss 44.11100720107555
loss 45.530630254149436
loss 46.94285540759564
loss 48.36171020925045
loss 49.795534334778786
Epoch:  8
training loss =

In [ ]:
'''
Load saved model from checkpoint
'''
model, optimizer, lr_scheduler, train_loss, v_loss, v_acc, epoch = load(modelpath, model, optimizer, lr_scheduler)


In [12]:
v_loss, top1_acc, accuracy_dict= eval_classify(model, validation_dataloader, criterion, device)
print('Validation Loss: {:.4f}\tTop 1 Validation Accuracy: {:.4f}\nAccuracy:{}\t'.format(v_loss, top1_acc, accuracy_dict))

Validation Loss: 0.6577	Top 1 Validation Accuracy: 0.8654
Accuracy:defaultdict(<class 'int'>, {'Top 1 Accuracy': 86.54392178672833, 'Top 5 Accuracy': 96.05128355238122, 'Top 10 Accuracy': 97.26604806369464, 'Top 20 Accuracy': 98.00954248748647, 'Top 30 Accuracy': 98.32860111816878, 'Top 50 Accuracy': 98.66814975265639, 'Top 100 Accuracy': 99.03404267775078})	
